## A Notebook to get Information from existing IIIF-Manifests, select canvases, and arrange them into a new manifest


NOTE: For now, this will only work with existing manifests following the v2 of the IIIF Presentation API, but will produce a manifest following v3.

In [43]:
import requests
import json

In [44]:
# function to create a new manifest
def create_manifest(*args):
   
    new_manifest = {
        "@context": "http://iiif.io/api/presentation/3/context.json",
        "id": manifest_id,
        "type": "Manifest",
        "label": label,
        "metadata": metadata,
        "provider": provider,
        "seeAlso": seeAlso,
        "items": []  # No canvases yet
    }  

    return new_manifest

In [45]:
# Provide basic Metadata for the new manifest
base_url="https://raw.githubusercontent.com/r0man-ist/iiif/refs/heads/main/Yd7829/"
filename ="Yd7829.json"
manifest_id=base_url+filename ## required

label={"de": ["Rekonstruktion des Sammelbandes Yd7829"]} ## required
metadata=[
   
    {
      "label":  {"de": ["Sprache"]},
      "value":  {"de": ["ger"]}
    },
     {
      "label":  {"de": ["Lizenz"]},
      "value":  {"de": ["Public Domain Mark 1.0"]}
    },
      {
      "label":  {"de": ["Anmerkung"]},
      "value":  {"de": ["Bei diesem virtuellen Objekt handelt es sich um eine Rekonstruktion des Sammelbandes mit der Signatur Yd7829, der seit 1929 an der Staatsbibliothek zu Berlin als vermisst gemeldet ist."]}
    }
    ]

provider = [
    {
      "id": "https://lab.sbb.berlin",
      "type": "Agent",
      "label": { "en": [ "Stabi Lab" ] },
      "homepage": [
        {
          "id": "https://lab.sbb.berlin",
          "type": "Text",
          "label": { "de": [ "Stabi Lab Homepage" ] },
          "format": "text/html"
        }]
    }      
  ]

seeAlso = [
    {
      "id": "https://cinquecentine.wordpress.com/2019/11/04/gestohlen-zerstueckelt-und-ueber-antiquariate-verkauft-geschichte-eines-konvoluts/",
      "type": "text",
      "label": {
        "de": [
          "Blogbeitrag"
        ]
      }
    }
]
    
new_manifest = create_manifest(manifest_id, label, metadata, provider, seeAlso)

In [46]:
# function to get manifests from a list of urls
def fetch_manifests(manifest_urls):
    manifests = []
    for url in manifest_urls:
        response = requests.get(url)
        response.raise_for_status()  # Raises an HTTPError for bad responses
        manifest = response.json()  # Parse the JSON response
                             
        manifests.append(manifest)
            
    return manifests

In [47]:
# specify a list of manifests to fetch
manifest_urls = ["https://collections.library.yale.edu/manifests/17384131", #2
                 "https://collections.library.yale.edu/manifests/17384177", #3
                 "https://raw.githubusercontent.com/r0man-ist/iiif/refs/heads/main/Yd7829/manifests/bsb00062258v3.manifest.json", #4
                 "https://collections.library.yale.edu/manifests/17384255", #5
                 ]
manifests = fetch_manifests(manifest_urls)
print(manifests)



[{'@context': ['http://iiif.io/api/search/1/context.json', 'http://iiif.io/api/extension/navplace/context.json', 'http://iiif.io/api/presentation/3/context.json'], 'id': 'https://collections.library.yale.edu/manifests/17384131', 'type': 'Manifest', 'label': {'none': ['Ein new Lied vom Türckenn : inn dem Thon, Vom Künig vonn Franckreich, was wöl wir aber heben an [et]c.']}, 'homepage': [{'id': 'https://collections.library.yale.edu/catalog/17384131', 'type': 'Text', 'format': 'text/html', 'label': {'en': ['Yale Digital Collections page']}}], 'requiredStatement': {'label': {'en': ['Provider']}, 'value': {'en': ['Yale University Library']}}, 'rendering': [{'id': 'https://collections.library.yale.edu/pdfs/17384131.pdf', 'type': 'Text', 'format': 'application/pdf', 'label': {'en': ['Download as PDF']}}], 'seeAlso': [{'id': 'https://collections.library.yale.edu/catalog/oai?verb=GetRecord&metadataPrefix=oai_mods&identifier=oai:collections.library.yale.edu:17384131', 'type': 'Dataset', 'form

In [48]:
# get individual canvases from the manifests
canvases = []
for manifest in manifests:  
    for canvas in manifest.get('items', []):
        if canvas.get('type') == 'Canvas':
            canvases.append(canvas)

In [49]:
# OPTIONAL
# specify a list of canvases to REMOVE in the new manifest
to_exclude_canvas_list = [
]


In [51]:
matching_canvases = []


# Function to remove canvases from the new manifest
matching_canvases = [canvas for canvas in canvases if canvas not in to_exclude_canvas_list]


In [52]:
# provide new URLs for the canvases

canvas_nr = 1
for canvas in matching_canvases:
    canvas["id"] = base_url + filename.split(".")[0] + "/" + str(canvas_nr) + "/" + "canvas"
    canvas_nr += 1



# provide new URLs for the annotation pages

for canvas in matching_canvases:
    for annotation in canvas["items"]:
        annotation["id"] = canvas["id"] + "/annotation-page"

# update the target URL for the annotation pages

for canvas in matching_canvases:
    for annotation in canvas["items"]:
        annotation["items"][0]["target"] = canvas["id"]

new_manifest["items"] = matching_canvases

In [53]:
# renumber canvas labels
canvas_nr = 1
for canvas in matching_canvases:
    canvas["label"].clear()  # Remove all other labels
    canvas["label"]["de"] = str(canvas_nr)
    canvas_nr += 1

new_manifest["items"] = matching_canvases

In [54]:

with open("Yd7829.json", "w", encoding="utf-8") as f:
        json.dump(new_manifest, f, ensure_ascii=False, indent=4)